In [1]:
# auto reload das libs
%load_ext autoreload 
%autoreload 2 

In [2]:
from hmmlearn import hmm
import pandas as pd
import numpy as np
from data_handler import DataHandler
import plotly.graph_objects as go

In [3]:
dhandler = DataHandler()
X = dhandler.prepare_gaussian_feature('daily_return')

In [19]:
model = hmm.GaussianHMM(n_components=3, covariance_type='full')
model.fit(X)
Z = model.predict(X)

In [20]:
ibov_df = dhandler.ibov_df
ibov_df.loc[:, 'state'] = Z
ibov_df.groupby('state')['daily_return'].describe()


,count,mean,std,min,25%,50%,75%,max
state,,,,,,,,
0,2043.0,-0.000899,0.010713,-0.029493,-0.008169,-0.000501,0.006838,0.020077
1,127.0,0.023868,0.003370,0.015649,0.021574,0.023222,0.025940,0.032331
2,146.0,-0.000945,0.044854,-0.147804,-0.032949,0.011417,0.036261,0.139089


In [36]:
ibov_df.loc[:, 'state_str'] = ibov_df['state'].apply(lambda x: f"s{x}")
areas = ibov_df[['date', 'state_str']].copy()
areas.loc[:, 'max_date'] = areas['date']
label_groups = areas['state_str'].ne(areas['state_str'].shift()).cumsum()
grpd_areas = (
    areas.groupby(label_groups).agg({'date':'min', 'max_date':'max', 'state_str':'last'})
           .reset_index(drop=True))

In [38]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=ibov_df['date'], y=ibov_df['closing_price'], mode='lines'))
for _, row in grpd_areas.iterrows():
    if row['state_str'] == 's0':
        color = 'yellow'
    elif row['state_str'] == 's1':
        color = 'green'
    else:
        color = 'red'

    fig.add_vrect(x0=row['date'], x1=row['max_date'], fillcolor=color, opacity=0.1, line_width=0)

fig.show()

In [7]:
model = hmm.GaussianHMM(n_components=3, covariance_type='full')
features = ['daily_return', 'weekly_return', 'monthly_return', 'volatility']
X2, lenghts = dhandler.prepare_multiple_gaussian_features(features)
model.fit(X2, lenghts)
Z = model.predict(X2, lenghts)

ValueError: Expected 2D array, got 1D array instead:
array=[4.01132464e-04 9.66511903e-03 9.65999871e-05 ... 1.66604359e-01
 1.69031318e-01 1.69196385e-01].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

AttributeError: 'tuple' object has no attribute 'reshape'